In [3]:
import pandas as pd

import modulo_producciones as prod

In [4]:
prod.ModuloProducciones()

In [26]:
INPUT_FILEPATH = "input/produccion_junio_2023.xlsx"
MES_A_ANALIZAR = "June"

In [57]:

# Este diccionario relaciona la hoja de los procedimientos a datos en el formato de SIGCOM.
# El primer elemento de la lista corresponde a la fila en donde debe ser cargado ese procedimiento.
# El segundo elemento corresonde a que CC de consulta debe ser cargado el procedimiento (en caso
# que tenga alguna consulta)
CORRELACION_PROCEDIMIENTOS_SIGCOM = {
    "QMT": ["PROCEDIMIENTO ONCOLOGÍA", "CONSULTA ONCOLOGÍA"],
    "APNEA": ["PROCEDIMIENTO DE NEUMOLOGÍA", "CONSULTA NEUMOLOGÍA"],
    "EF": ["PROCEDIMIENTOS DE CARDIOLOGÍA", ""],
    "EBUS": ["PROCEDIMIENTOS EBUS", "CONSULTA NEUMOLOGÍA"],
    "TAVI": ["PROCEDIMIENTOS TAVI", ""],
    "ASIST VENTRICULAR": ["PROCEDIMIENTOS ASISTENCIA VENTRICULAR", ""],
    "ECMO": ["PROCEDIMIENTOS ECMO", ""],
    "HMD": ["PROCEDIMIENTOS DE HEMODINAMIA", "CONSULTA CARDIOLOGÍA"],
    "BRONCOSCOPIA": ["PROCEDIMIENTOS ENDOSCÓPICOS", "CONSULTA NEUMOLOGÍA"],
}

SERVICIO_INTERNO_SIGCOM = {
    "ambulatorio": "CONSULTAS",
    "medico quirugico 3° norte y sur": "HOSPITALIZACIÓN MEDICINA INTERNA/HOSPITALIZACIÓN QUIRÚRGICA",
    "medico quirugico 4° norte": "HOSPITALIZACIÓN CARDIOVASCULAR",
    "uci 4° sur": "UNIDAD DE CUIDADOS INTENSIVOS",
    "uci 5° norte": "UNIDAD DE CUIDADOS INTENSIVOS",
    "uci 5° sur": "UNIDAD DE CUIDADOS INTENSIVOS",
    "uti 3° norte": "UNIDAD DE TRATAMIENTO INTENSIVO ADULTO",
    "uti 4° norte": "UNIDAD DE TRATAMIENTO INTENSIVO ADULTO",
    "uti 4° sur": "UNIDAD DE TRATAMIENTO INTENSIVO ADULTO",
    "uti 5° norte": "UNIDAD DE TRATAMIENTO INTENSIVO ADULTO",
}

In [53]:
def obtener_localizacion_de_procedimientos(ruta_produccion):
    resultados = {}
    archivo = pd.ExcelFile(ruta_produccion)
    for hoja in archivo.sheet_names[3:]:
        df = pd.read_excel(archivo, sheet_name=hoja)
        df = df.loc[:, "Rut":"Hoja1.Nombre nuevo HIS"]
        df["Hoja1.Nombre nuevo HIS"] = df["Hoja1.Nombre nuevo HIS"].str.strip().str.lower()
        df["SIGCOM"] = df["Hoja1.Nombre nuevo HIS"].replace(SERVICIO_INTERNO_SIGCOM)

        procedimientos_mes = df[df["Fecha Realización"].dt.month_name() == MES_A_ANALIZAR]
        procedimientos_por_servicio = (
            procedimientos_mes.groupby("SIGCOM")["Rut"]
            .count()
            .reset_index(name="cantidad_procedimientos")
        )

        resultados[hoja] = procedimientos_por_servicio

    return resultados
    

In [54]:
resultados_procedimientos = obtener_localizacion_de_procedimientos(INPUT_FILEPATH)

In [56]:
resultados_procedimientos["QMT"]

,SIGCOM,cantidad_procedimientos
0,CONSULTAS,134
1,HOSPITALIZACIÓN MEDICINA INTERNA/HOSPITALIZACI...,5
